In [4]:
import os
import sys
import csv
import json
from datetime import datetime
from datetime import timedelta
import numpy as np
import pandas as pd
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [ ]:
class BasicFuncs():
    def __init__(self, path):
        self.root_dir = path
#         self.write_dir = write
      
    def mylistdir(self, directory, bit='', end=True):
        filelist = os.listdir(directory)
        if end:
            return [x for x in filelist if x.endswith(f'{bit}') and not x.startswith('.') and not 'Icon' in x]
        else:
             return [x for x in filelist if x.startswith(f'{bit}') and not x.startswith('.') and not 'Icon' in x]

    def make_storage_directory(self, target_dir):
        if not os.path.exists(target_dir):
            os.makedirs(target_dir)
        return target_dir

In [232]:
class TempstoDF():
    def __init__(self, path, w_name):
#         BasicFuncs.__init__(self, path)
        self.root_dir = path
        self.weather_file = os.path.join(self.root_dir, w_name)
        
        
        
    def read_T(self, file_p):
        df = pd.read_csv(file_p)
        df.Temperature_degC = df.Temperature_degC.round()
        df.Hour = pd.to_datetime(df.Hour).dt.strftime('%H:%M')
        return df
    
    def get_dist(self, df):
        all_H = df.Hour.unique()
        max_diff = df['Temperature_degC'].diff(periods=-1).max()
        min_diff = df['Temperature_degC'].diff(periods=-1).min()
        cols = [t for t in range(int(min_diff), int(max_diff)+1)]
        
        Temp_T = {}
        for h in all_H:
            p = []
            for t in range(int(min_diff), int(max_diff)+1):
                p.append(len(df.loc[(df['Temperature_degC'].diff(periods=-1)==t) & (df['Hour'] == h)]))
            new_p = [i/sum(p) for i in p]
            Temp_T[h] = new_p
        self.write_csv(Temp_T, cols)
        return Temp_T
 
    
    def write_csv(self, dist_dict, col_names):
        csv_file = os.path.join(self.root_dir, 'To-transitions.csv')
        with open(csv_file, 'w') as f:
            f.write(','+f'{cols}'.strip("[]")+'\n')
            for hour in dist_dict:
                f.write(f'{hour}' +','+ f'{dist_dict[hour]}'.strip("[]") + '\n')

    def main(self):
        df = self.read_T(self.weather_file)
        self.dist = self.get_dist(df)

In [134]:
class TempsOtoDF():
    def __init__(self, r_path, w_name):
        self.root_dir = r_path
        self.weather_file = os.path.join(self.root_dir, w_name)
        
    def read_T(self, file_p):
        df = pd.read_csv(file_p)
        df.Temperature_degC = df.Temperature_degC.round()
        df.Hour = pd.to_datetime(df.Hour).dt.strftime('%H:%M')
        return df
    
    def ind(self, t):
        return int(t-self.minT)
    
    def get_dist(self, df):
        all_H = df.Hour.unique()
        self.minT = df['Temperature_degC'].min()
        self.maxT = df['Temperature_degC'].max()
        allT = [t for t in range(int(minT), int(maxT)+1)]
        self.write_allT(allT)
        df['Tprime'] = df['Temperature_degC'].shift(-1)
        df = df.drop(df.tail(1).index)
        Temp_T = {}
        for h in all_H:
            transT_h = np.zeros((len(allT), len(allT)))
            for index, row in df[df.Hour == h].iterrows():
                i, j = self.ind(row['Temperature_degC']), self.ind(row['Tprime'])
                transT_h[i,j] += 1
            transT_h = transT_h/transT_h.sum()
            Temp_T[h] = transT_h
        return Temp_T
        
    def write_allT(self, allT):
        T_ind = [self.ind(t) for t in allT]
        csv_file = os.path.join(self.root_dir, f'temperature_indices.csv')
        np.savetxt(csv_file, np.column_stack((T_ind, allT)), fmt='%10.1f', delimiter=',')
    
    def write_csv(self, dist_dict):#, cols):
        for h in dist_dict:       
            csv_file = os.path.join(self.root_dir, f'To_{h[0:2]}.csv')
            np.savetxt(csv_file, dist_dict[h], fmt='%10.5f', delimiter=',')

            
    def main(self):
        self.df = self.read_T(self.weather_file)
        self.T = self.get_dist(df)
        self.write_csv(self.T)


In [133]:
read_path = '/Users/maggie/Documents/Github/DMU-Final-Project/boulder_hourlyTemps_jan.csv'
write_path = '/Users/maggie/Documents/Github/DMU-Final-Project/'
weather_file = 'boulder_hourlyTemps_jan.csv'
t = TempsOtoDF(write_path, weather_file)
t.main()


# print(test_df.head())
# print(len(df.loc[df['Temperature(degC)']<(df['Temperature(degC)'].mean()-2*df['Temperature(degC)'].std())]))
# print(len(df.loc[df['Temperature(degC)']>(df['Temperature(degC)'].mean()+2*df['Temperature(degC)'].std())]))

In [321]:
class TiTransitionsCSV():
    def __init__(self, r_path):
        self.root_dir = r_path
        
    def mylistdir(self, directory, bit='', end=True):
        filelist = os.listdir(directory)
        if end:
            return [x for x in filelist if x.endswith(f'{bit}') and not x.startswith('.') and not 'Icon' in x]
        else:
             return [x for x in filelist if x.startswith(f'{bit}') and not x.startswith('.') and not 'Icon' in x]

    def make_storage_directory(self, target_dir):
        if not os.path.exists(target_dir):
            os.makedirs(target_dir)
        return target_dir        
          
        
    def read_T(self, file_p):
        df = pd.read_csv(file_p)
        columns = ['Ti', 'Ti_prime', 'To']
        for col in columns:
            df[col] = df[col].round()
        df['To_prime'] = df['To'].shift(-1)
        df = df.drop(df.tail(1).index)
        return df

    
    def get_dist(self, df, rf_num):
        minTi, maxTi = df['Ti'].min(), df['Ti'].max()
        minTo, maxTo = df['To'].min(), df['To'].max()
        allTi = [t for t in range(int(minTi), int(maxTi)+1)]
        allTo = [t for t in range(int(minTo), int(maxTo)+1)]
        
#         self.allTi = [t for t in range(int(minTi), int(maxTi)+1)]
#         self.allTo = [t for t in range(int(minTo), int(maxTo)+1)]
#         self.all_pairs = [(x,y) for x in self.allTo for y in self.allTi]
#         self.inds_dict = {x:i for x,i in enumerate(self.all_pairs)}

        
        all_pairs = [(x,y) for x in allTo for y in allTi]
        inds = {x:i for i, x in enumerate(all_pairs)}
        self.write_inds(inds, rf_num)
    
        Temp_T = {}
        transT = np.zeros((len(allTi)*len(allTo), len(allTi)*len(allTo)))
        for index, row in df.iterrows():


            i = inds[(int(row.To), int(row.Ti))]
            j = inds[(int(row.To_prime), int(row.Ti_prime))]            
            transT[i,j] += 1
          
        transT = transT/transT.sum()
        return transT


    def write_inds(self, allT, rf):
        csv_file = os.path.join(self.root_dir,  f'pair_inds-{rf}.csv')
        np.savetxt(csv_file, np.column_stack(([k for k in inds_dict.keys()], [v for v in inds_dict.values()])), 
                   fmt='%10.1f', delimiter=',', header='index,temp out,temp in', comments='')

    
    def write_csv(self, transitionT, rf_num):    
        
        T_store = self.make_storage_directory(os.path.join(self.root_dir, 'TransitionProbabilties'))
        csv_file = os.path.join(T_store, f'To_Ti-{rf_num}.csv')
        np.savetxt(csv_file, transitionT, fmt='%10.5f', delimiter=',')

            
    def main(self):
        for f in self.mylistdir(os.path.join(self.root_dir, 'data_from_simulink'), bit='.csv'):
            rf_num = f.split('_')[1].strip('.csv')
            weather_file = os.path.join(self.root_dir, 'data_from_simulink', f)
            self.df = self.read_T(weather_file)
            T = self.get_dist(df, rf_num)
            self.write_csv(T, rf_num)


In [322]:
root_dir = '/Users/maggie/Desktop/DMU_project_test/'
t = TiTransitionsCSV(root_dir)
t.main()
df = t.df




# print(t.minTi, t.minTip, t.minTo)
# print(t.maxTi, t.maxTip, t.maxTo)

In [264]:
Temp_T = {}

transT_h = np.zeros((len(t.allTi)*len(t.allTo), len(t.allTi)*len(t.allTo)))
for index, row in df.iterrows():
    print(index)
    print(row)
    x = (row.Ti, row.To)
    print(x)
    break

0
Ti            22.0
Ti_prime      22.0
To            -5.0
a           6000.0
To_prime      -4.0
Name: 0, dtype: float64
(22.0, -5.0)


In [384]:
toy1 = pd.DataFrame(np.array([[1, 2], [2,1], [3,3], [3,1], [2,2], [1,3]]), columns=['t1_prime', 't2_prime'], index=['T1t1','T1t2', 'T2t1', 'T2t2', 'T3t1', 'T3t2' ])

toy2 = pd.DataFrame(np.array([[1, 2, 3], [2,1, 1], [3,3, 3]]), columns=['T1_prime', 'T2_prime', 'T3_prime'], index=['T1', 'T2', 'T3' ])
print("original matrices")
print(toy1)
print(toy2)

toy1_dup = toy1
toy2_dup = toy2

# toy1_dup.insert(1, 't1_prime2', toy1['t1_prime'])


for col in toy2.columns:
    for i in reversed(range(1, len(toy1.columns))):
        toy3 = toy2_dup.insert(toy2.columns.get_loc(col), f'{col}-{i}', toy2[col])       
toy3 = toy2_dup.iloc[np.arange(len(toy2_dup)).repeat(2)]
# print(toy3)

    # toy2_dup2 = pd.DataFrame(np.repeat(toy2_dup.values,2,axis=0))
# toy2_dup2.columns = toy2_dup.columns




for i in range(1, len(toy2)):
    toy1_dup = pd.concat([toy1_dup, toy1], axis=1, sort=False)

print("extended matrices")    
print(toy1_dup)
print(toy3)


original matrices
      t1_prime  t2_prime
T1t1         1         2
T1t2         2         1
T2t1         3         3
T2t2         3         1
T3t1         2         2
T3t2         1         3
    T1_prime  T2_prime  T3_prime
T1         1         2         3
T2         2         1         1
T3         3         3         3
extended matrices
      t1_prime  t2_prime  t1_prime  t2_prime  t1_prime  t2_prime
T1t1         1         2         1         2         1         2
T1t2         2         1         2         1         2         1
T2t1         3         3         3         3         3         3
T2t2         3         1         3         1         3         1
T3t1         2         2         2         2         2         2
T3t2         1         3         1         3         1         3
    T1_prime-1  T1_prime  T2_prime-1  T2_prime  T3_prime-1  T3_prime
T1           1         1           2         2           3         3
T1           1         1           2         2           3      